In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
%matplotlib inline

In [2]:
def swc_to_dataframe( file_name ) :
    col_names = ['id', 'type', 'x', 'y', 'z', 'radius', 'parent', 'seg_id', 'level', 'mode', 'timestamp', 'TFresindex']
    df = pd.read_csv( file_name, sep=r"\s+", comment = "#", names = col_names, index_col = 'id')
    return df

In [3]:
def filter_swc( df, type ) :
    dd = df[df['type']==type]
    return dd

In [4]:
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST"

# Image file is downsampled tif with proper spacing headers (no origin or orientation)
image_directory = os.path.join( model_directory, "BIL_DATA/proj/u19zeng/fMOST_downsampled", "fixed_headers")

# SWC files with coordinates matching full resolution image index
sample_id = '192343'
swc_directory = os.path.join( model_directory, "HUSTReproduction_9_7_2021", "Sandbox", "HUST_REGISTRATION_AllCells_OriginalSent", sample_id )
output_directory = os.path.join( model_directory, "HUSTReproduction_9_7_2021","Sandbox", "HUST_REGISTRATION_AllCells_OriginalSent_thumbnail", sample_id)

In [5]:
#
# component types
#
types = {}
types['soma'] = {'type_id': 1, 'color': 'black'}
types['axon'] = {'type_id': 2, 'color': 'cyan'}
types['basal dendrite'] = {'type_id': 3, 'color': 'orangered'}
types['apical dendrite'] = {'type_id': 4, 'color': 'maroon'}

#
# views for swc
#
views = {}
views['coronal'] = {'order': 0, 'x': 'y', 'y': 'x'}
views['sagittal'] = {'order': 1, 'x': 'z', 'y' : 'x'}
views['horizontal'] = {'order': 2, 'x': 'z', 'y' : 'y'}

#
# image slices
#
slices = {}
slices['coronal'] = {'swc': 'z', 'image' : 'z', 'transpose' : True }
slices['sagittal'] = {'swc': 'y', 'image' : 'y', 'transpose' : True }
slices['horizontal'] = {'swc' : 'x', 'image' : 'x', 'transpose' : True }

#
# scale from image pixel index to SWC coordinate
#
scales = {}
scales['z'] = 8
scales['x'] = 32
scales['y'] = 32

In [6]:
#
# Create morphology thumbnail for each SWC file in the swc directory
#
#
for swc_file in glob.glob(os.path.join(swc_directory, "*.swc")):
    
    print(swc_file)
    
    bb = os.path.basename( swc_file )
    
    # find the associated brain file
    bname = bb.split('_')[0]
    image_file = os.path.join(image_directory, "mouse%sgreen_xy32z8.nii.gz" % bname )
    
    if not os.path.isfile( image_file ) :
        continue
    
    # set the output thumbnail png file name
    cname = bb.split('.')[0]
    thumnail_file = os.path.join( output_directory, "%s.png" % cname )
    
    print((cname,bname))
    
    #
    # read swc components into dataframe
    #
    df = swc_to_dataframe( swc_file )

    #
    # read in image file
    #
    image = sitk.ReadImage( image_file )
    print(image.GetSize())
    
    vmax = np.quantile(sitk.GetArrayViewFromImage( image ),[0.95])[0] * 2.0


    #
    # scale swc coordinates to match image volume
    #
    for sk, sv in scales.items() :
        df[sk] = df[sk] / sv


    #
    # get background image from soma location
    #

    filtered = filter_swc(df,types['soma']['type_id'])
    
    if len(filtered) == 0 :
        print("no soma found")
        continue
        
    soma = filtered.iloc[0]


    #
    #  plot swc components onto of image at soma location
    #
    plt.rcParams["figure.figsize"] = (30,8)

    fig, axs = plt.subplots(1,3)
    fig.suptitle(os.path.basename(swc_file))

    for vk, vv in views.items() :

        idx = int(round(soma[slices[vk]['swc']],0))

        if slices[vk]['image'] == 'x' :
            bkg = sitk.GetArrayViewFromImage( image )[:,:,idx]
        elif slices[vk]['image'] == 'y' :
            bkg = sitk.GetArrayViewFromImage( image )[:,idx,:]
        elif slices[vk]['image'] == 'z' :
            bkg = sitk.GetArrayViewFromImage( image )[idx,:,:]

        if slices[vk]['transpose'] :
            bkg = bkg.transpose()

        axs[vv['order']].imshow( bkg, cmap='gray', vmax=vmax)

        for tk,tv in types.items() :

            filtered = filter_swc(df, tv['type_id'])

            x = filtered[vv['x']].values
            y = filtered[vv['y']].values

            axs[vv['order']].scatter( x, y, s=0.001, c=tv['color'])

        # TO DO: fix aspect ratio
        axs[vv['order']].set_aspect('equal')

    
    plt.savefig(thumnail_file, dpi = 150)
    plt.close("all")
    

C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST\HUSTReproduction_9_7_2021\Sandbox\HUST_REGISTRATION_AllCells_OriginalSent\192343\192343_3226-X6435-Y14389.swc
('192343_3226-X6435-Y14389', '192343')
(591, 962, 1464)
C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST\HUSTReproduction_9_7_2021\Sandbox\HUST_REGISTRATION_AllCells_OriginalSent\192343\192343_3481-X7225-Y16596.swc
('192343_3481-X7225-Y16596', '192343')
(591, 962, 1464)
C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST\HUSTReproduction_9_7_2021\Sandbox\HUST_REGISTRATION_AllCells_OriginalSent\192343\192343_3985-X6283-Y20312.swc
('192343_3985-X6283-Y20312', '192343')
(591, 962, 1464)
C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST\HUSTReproduction_9_7_2021\Sandbox\HUST_REGISTRATION_AllCells_OriginalSent\192343\192343_5011-X4797-Y11341.swc
('192343_5011-X4797-Y11341', '192343')
(591, 962, 1464)
C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST\HUSTReproduction_9_7_2021\Sandbox\HUST_REGISTRATION_AllC